# NS delayes notification

NS API address: https://www.ns.nl/en/travel-information/ns-api

credentials management: https://dev.to/jamestimmins/django-cheat-sheet-keep-credentials-secure-with-environment-variables-2ah5
https://pypi.org/project/python-dotenv/


In [1]:
#!pip install python-dotenvb
#!pip install osa

In [2]:
#pip install jupyter_helpers

In [177]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, pandas as pd
import json
import os
from pandas.io.json import json_normalize
import numpy as np
from jupyter_helpers.namespace import NeatNamespace
import datetime
import time
import requests
import re
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from collections import defaultdict

pd.set_option('display.max_columns', 300) # display 300 columns without shrinking
pd.set_option('display.max_rows', 100) # Show more rows

In [178]:
from dotenv import load_dotenv
load_dotenv(override=True)
# from pathlib import Path  # python3 only using specific path
# env_path = Path('.') / '.env'
# load_dotenv(dotenv_path=env_path)

True

## Manual data insert

In [179]:
user_data=defaultdict()
user_data['daily_work_leave'] = {'from_uic': 8400061, 'to_uic': 8400319, 'h':6, 'm':26, 'enabled':True}
user_data['daily_work_return'] = {'from_uic': 8400319, 'to_uic': 8400061, 'h':16, 'm':8, 'enabled':True}

In [180]:
df_user_data = pd.DataFrame(user_data).T
df_user_data[df_user_data['enabled']==True]
df_user_data['search_date_time'] = df_user_data.apply(lambda row: datetime.datetime.today().replace(hour=row.h, minute=row.m, second =0).strftime('%Y-%m-%dT%T%z') ,axis=1)
df_user_data['current_date_time'] = datetime.datetime.today()
df_user_data['diff'] = pd.to_datetime(df_user_data['search_date_time']) - pd.to_datetime(df_user_data['current_date_time'])
df_user_data_input = df_user_data[df_user_data['diff']>pd.Timedelta(0, unit='s')].sort_values(by='diff').head(1).to_dict(orient='records')

In [182]:
# len(df_user_data_input)

# Logic

## Calculation
1. check if train is canceled
    * Canceled train return next earliest available one
    * And calculate waiting time
2. check if train is delayed
    * Calculate

In [53]:
# #Set UIC code, stations code can be googled with universal destinations.
# denbosch = 8400319
# amszuid = 8400061 
# search_date_time_leave = datetime.datetime.today().replace(hour=6, minute=26, second =0).strftime('%Y-%m-%dT%T%z')
# search_date_time_return = datetime.datetime.today().replace(hour=16, minute=8, second =0).strftime('%Y-%m-%dT%T%z')
# from_uic = amszuid
# to_uic = denbosch
# email_sent = False
#train_id = 'IC 3519 ' # Return 3556
#ctx_Reconn_amsz_to_denbosch = 'arnu|fromStation=8400061|toStation=8400319|plannedFromTime=2019-12-23T06:26:00+01:00|plannedArrivalTime=2019-12-23T07:21:00+01:00|yearCard=false|excludeHighSpeedTrains=false'

In [183]:
def get_data_from_ns(from_uic, to_uic, search_date_time, email_sent, **kwargs):
    # all ride information
    # https://apiportal.ns.nl/docs/services/public-reisinformatie-api/operations/ApiV3TripsGet?

    key = os.getenv("NS_KEY")

    headers = {
        # Request headers
        'Accept': '',
        'X-Request-ID': '',
        'X-Caller-ID': '',
        'x-api-key': '',
        'Authorization': '',
        'Ocp-Apim-Subscription-Key': '%s' % key,
    } 

    params = urllib.parse.urlencode({
        # Request parameters
        'originUicCode': '%s' % from_uic,
        'destinationUicCode': '%s' % to_uic,
        'dateTime': '%s' % search_date_time,
        'previousAdvices': 2,
        'nextAdvices': 8 ,
    })

    try:
        conn = http.client.HTTPSConnection('gateway.apiportal.ns.nl')
        conn.request("GET", "/public-reisinformatie/api/v3/trips?%s" % params, "{body}", headers)
        response = conn.getresponse()
        data = response.read()
        #print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    def date_time_column(x):
        date_time_col=[]
        l = list(x.columns)
        for i in l:
            if i.find('DateTime')==-1:
                continue
            else: 
                date_time_col.append(i)
                x[i] = pd.to_datetime(x[i])
        return x    
    
    
    d = json.loads(data)
    lenghth = len(d)
    if lenghth >0:
        print(f'Data successfully got from NS. len={lenghth}')
    
    df_unpack = json_normalize(data=d['trips'], record_path=['legs'], meta=['ctxRecon','status','transfers','plannedDurationInMinutes','actualDurationInMinutes','punctuality'
                                                                       , 'realtime','optimal','shareUrl'], errors='ignore',meta_prefix='meta_' , sep='_')
    #print(type(df_unpack['origin_plannedDateTime']))
    
    date_time_column(df_unpack)
    
    df_unpack['time_to_target'] = df_unpack['origin_plannedDateTime'].apply(lambda x: abs(x - pd.Timestamp(search_date_time+'+0100') ) )
    #print(df_unpack['time_to_target'])
    
    #target_train = df_unpack.copy()[df_unpack['origin_plannedDateTime']==search_date_time+'+0100'].head(1)
    target_train = df_unpack.copy().sort_values(by='time_to_target', ascending =True).head(1)
    selected_len = len(target_train)

    
    #date_time_column(target_train)
    
    def status(target_train):
        status='normal'
        delayed_min = 0 
        url_to_html = target_train['meta_shareUrl'].values.item()['uri']
        if target_train['cancelled'].bool() == True:
            status = 'cancelled'
        else:
            try:
                diff = target_train['origin_actualDateTime'] - target_train['origin_plannedDateTime']
                delayed_min = diff.values.item()/60000000000
                if delayed_min > 5 :
                    status = 'delayed > 5 mins'
                else:
                    status = 'delayed <= 5 mins'
            except:
                if target_train['meta_status'].values.item() != 'NORMAL':
                    status = target_train['meta_status'].values.item()
                else: 
                    status = 'normal'
            pass

        return status, int(delayed_min), url_to_html
    
    if selected_len>0:
        print(f'Target time selected. Time={search_date_time}')
        status, delayed_min, url_to_html = status(target_train)
        print(f'Status {status}; Delayed {delayed_min} mins.')
    else:
        print(f'No train selected for time {search_date_time}')
        
    def send_email(status, delayed_min, url_to_html ):

        sender_email = os.getenv('GMAIL_USER')
        receiver_email = [i.strip() for i in list(os.getenv("SEND_TO").split(";"))][1]
        password = os.getenv('GMAIL_PWD')

        message = MIMEMultipart("alternative")
        message["Subject"] = 'Train '+status+'!'
        message["From"] = sender_email
        message["To"] = receiver_email

        # Create the plain-text and HTML version of your message
        text = """\
        Hi,
        """ + url_to_html
        html = '' #requests.get(url_to_html).text 

        # Turn these into plain/html MIMEText objects
        part1 = MIMEText(text, "plain")
        part2 = MIMEText(html, "html")

        # Add HTML/plain-text parts to MIMEMultipart message
        # The email client will try to render the last part first
        message.attach(part1)
        message.attach(part2)

        # Create secure connection with server and send email
        context = ssl.create_default_context()
        
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
            server.login(sender_email, password)
            server.sendmail(
                sender_email, receiver_email, message.as_string()
            )   
    
    if status == 'normal':
        pass
    else:
        try:
            send_email(status, delayed_min, url_to_html)
            email_sent = True
        except:
            email_sent = False
    
    return email_sent

In [184]:
def check_which_job_to_run(df_user_data_input):
    if len(df_user_data_input)>0:
        try:
            email_sent = get_data_from_ns( email_sent=email_sent, **df_user_data_input[0])
        except:
            print('error: email not sent')

In [161]:
# email_sent = get_data_from_ns( to_uic, from_uic, search_date_time_return, email_sent)

In [162]:
# search_date_time_return
# time.strptime(search_date_time_return, '%Y-%m-%dT%H:%M:%S')
#pd.Timestamp(search_date_time_return)

In [25]:
#d

In [9]:
#NeatNamespace(d['trips'][0])

In [10]:
#df_trips.columns

In [11]:
#df_trips.info()

In [12]:
# df_unpack = json_normalize(data=d['trips'], record_path=['legs'], meta=['ctxRecon','status','transfers','plannedDurationInMinutes','actualDurationInMinutes','punctuality'
#                                                                        , 'realtime','optimal','shareUrl'], errors='ignore',meta_prefix='meta_' , sep='_')

In [13]:
#df_unpack.head()

In [14]:
#df_unpack['meta_shareUrl'][0]['uri']

In [15]:
#df_unpack[df_unpack['cancelled']==False]

In [16]:
# target_train = df_unpack.copy()[df_unpack['origin_plannedDateTime']==search_date_time+'+0100'].head(1)

In [28]:
#target_train = df_unpack.copy().head(1)

In [29]:
# def date_time_column(x):
#     date_time_col=[]
#     l = list(x.columns)
#     for i in l:
#         if i.find('DateTime')==-1:
#             continue
#         else: 
#             date_time_col.append(i)
#             x[i] = pd.to_datetime(x[i])
#     return x

In [30]:
# date_time_column(target_train)

,idx,name,travelType,direction,cancelled,changePossible,alternativeTransport,journeyDetailRef,notes,messages,stops,steps,crowdForecast,punctuality,crossPlatformTransfer,shorterStock,journeyDetail,reachable,origin_type,origin_prognosisType,origin_plannedTimeZoneOffset,origin_plannedDateTime,origin_plannedTrack,origin_checkinStatus,origin_notes,origin_name,origin_lng,origin_lat,origin_countryCode,origin_uicCode,origin_weight,origin_products,destination_type,destination_prognosisType,destination_plannedTimeZoneOffset,destination_plannedDateTime,destination_plannedTrack,destination_exitSide,destination_checkinStatus,destination_notes,destination_name,destination_lng,destination_lat,destination_countryCode,destination_uicCode,destination_weight,destination_products,product_number,product_categoryCode,product_shortCategoryName,product_longCategoryName,product_operatorCode,product_operatorName,product_type,product_displayName,origin_actualTimeZoneOffset,origin_actualDateTime,destination_actualTimeZoneOffset,destination_actualDateTime,meta_ctxRecon,meta_status,meta_transfers,meta_plannedDurationInMinutes,meta_actualDurationInMinutes,meta_punctuality,meta_realtime,meta_optimal,meta_shareUrl
0,0,SPR 5710,PUBLIC_TRANSIT,Hoofddorp,False,True,False,1|310166|0|884|18012020,"[{'value': '+eas, source RAST attributonstatio...",[],[{'notes': [{'value': 'Station Accessible or T...,[],LOW,100.0,False,False,"[{'type': 'TRAIN_XML', 'link': {'uri': 'https:...",False,STATION,PROGNOSED,60,2020-01-18 06:11:00+01:00,3,CHECKIN,[{'value': 'Station Accessible or Travel Assis...,Amsterdam Zuid,4.873061,52.339027,NL,8400061,-1,-1,STATION,PROGNOSED,60,2020-01-18 06:17:00+01:00,3,RIGHT,NOTHING,[{'value': 'Station Accessible or Travel Assis...,Schiphol Airport,4.762269,52.308293,NL,8400561,-1,-1,5710,SPR,SPR,Sprinter,ns,NS,TRAIN,NS Sprinter,NaN,NaT,NaN,NaT,arnu|fromStation=8400061|toStation=8400319|pla...,CANCELLED,2,97,97,90.9,True,False,{'uri': 'https://www.ns.nl/rpx?ctx=arnu%7Cfrom...


In [20]:
#diff = target_train['origin_actualDateTime'] - target_train['origin_plannedDateTime']

In [21]:
# diff.values.item()/60000000000

In [36]:
# def status(target_train):
#     status='normal'
#     delayed_min = 0 
#     url_to_html = target_train['meta_shareUrl'].values.item()['uri']
#     if target_train['cancelled'].bool() == True:
#         status = 'cancelled'
#     else:
#         try:
#             diff = target_train['origin_actualDateTime'] - target_train['origin_plannedDateTime']
#             delayed_min = diff.values.item()/60000000000
#             if delayed_min > 5 :
#                 status = 'delayed > 5 mins'
#             else:
#                 status = 'delayed <= 5 mins'
#         except:
#             if target_train['meta_status'].values.item() != 'NORMAL':
#                 status = target_train['meta_status'].values.item()
#             else: 
#                 status = 'normal'
#         pass
    
#     return status, int(delayed_min), url_to_html

In [37]:
# status, delayed_min, url_to_html = status(target_train)

In [38]:
# status

'CANCELLED'

In [34]:
# delayed_min

0

In [35]:
# url_to_html

'https://www.ns.nl/rpx?ctx=arnu%7CfromStation%3D8400061%7CtoStation%3D8400319%7CplannedFromTime%3D2020-01-18T06%3A11%3A00%2B01%3A00%7CplannedArrivalTime%3D2020-01-18T07%3A48%3A00%2B01%3A00%7CyearCard%3Dfalse%7CexcludeHighSpeedTrains%3Dfalse'

## Notification

In [ ]:
# import smtplib

# gmail_user = os.getenv('GMAIL_USER')
# gmail_password = os.getenv('GMAIL_PWD')
# server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
# server.ehlo()
# server.login(gmail_user, gmail_password)

In [ ]:
#txt_html = requests.get(url_to_html).text
#urllib.request.urlopen(url_to_html).read().decode("utf-8") 

In [ ]:
#<div class="rp-reisadvies"></reisadvies>

In [ ]:
# import smtplib, ssl
# from email.mime.text import MIMEText
# from email.mime.multipart import MIMEMultipart

# def send_email(status, delayed_min, url_to_html ):

#     sender_email = os.getenv('GMAIL_USER')
#     receiver_email = [i.strip() for i in list(os.getenv("SEND_TO").split(";"))][1]
#     password = os.getenv('GMAIL_PWD')

#     message = MIMEMultipart("alternative")
#     message["Subject"] = 'Train '+status+'!'
#     message["From"] = sender_email
#     message["To"] = receiver_email

#     # Create the plain-text and HTML version of your message
#     text = """\
#     Hi,
#     """ + url_to_html
#     html = '' #requests.get(url_to_html).text 

#     # Turn these into plain/html MIMEText objects
#     part1 = MIMEText(text, "plain")
#     part2 = MIMEText(html, "html")

#     # Add HTML/plain-text parts to MIMEMultipart message
#     # The email client will try to render the last part first
#     message.attach(part1)
#     message.attach(part2)

#     # Create secure connection with server and send email
#     context = ssl.create_default_context()
#     with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
#         server.login(sender_email, password)
#         server.sendmail(
#             sender_email, receiver_email, message.as_string()
#         )   

In [ ]:
# def notification():
#     message = ''
#     if 
#     elif 'origin.actualDateTime' in intersection:
#         m = 'Delayed by' + 
#         #print('Delayed')
    
#     else:
#         m = 'OK'
#         #print('OK')
#     return m

In [ ]:
# notification()
# m

## Save to S3

In [ ]:
# def df_to_s3(df, table_name, schema, dtype=None, sep='\t', delete_first=False, clean_df=False, keep_csv=True, chunksize=10000, if_exists="fail", redshift_str=None, bucket=None):

#     """Copies a dataframe inside a Redshift schema.table
#         using the bulk upload via this process:
#         df -> local csv -> s3 csv -> redshift table

#         NOTE: currently this function performs a delete * in
#         the target table, append is in TODO list, also we
#         need to add a timestamp column

#         COLUMN TYPES: right now you need to do a DROP TABLE to
#         change the column type, this needs to be changed TODO

#     Parameters
#     ----------
#     keep_csv : bool, optional
#         Whether to keep the local csv copy after uploading it to Amazon S3, by default True
#     redshift_str : str, optional
#         Redshift engine string, if None then 'mssql+pyodbc://Redshift'
#     bucket : str, optional
#         Bucket name, if None then 'teis-data'
#     """

#     ACCESS_KEY = os.getenv('AWS_ACCESS')
#     SECRET_KEY = os.getenv('AWS_SECRET')
#     REGION = os.getenv('AWS_REGION')

#     redshift_str = redshift_str if redshift_str else 'mssql+pyodbc://Redshift'
#     bucket_name = bucket if bucket else 'bing'

#     engine = create_engine(redshift_str, encoding='utf8', poolclass=NullPool)

#     s3 = boto3.resource('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY, region_name=REGION)
#     bucket = s3.Bucket(bucket_name)

#     filename = table_name + '.csv'
#     filepath = os.path.join(os.getcwd(), filename)

#     if delete_first:
#         remove_from_s3(table_name)
#         s3_file = s3.Object(bucket_name, filename)
#         s3_file.delete()

#     if clean_df:
#         df = df_clean(df)

#     df = clean_colnames(df)
#     df.columns = df.columns.str.strip().str.replace(" ", "_") # Redshift won't accept column names with spaces

#     df.to_csv(filepath, sep="\t", encoding="utf-8", index=False, chunksize=chunksize)
#     print(f'{filename} created in {filepath}')

#     bucket.upload_file(filepath, f"bulk/{filename}")
#     print(f'bulk/{filename} file uploaded to s3')

#     if check_if_exists(table_name, schema, redshift_str=redshift_str):
#         if if_exists == 'fail':
#             raise ValueError(f"Table {table_name} already exists")
#         elif if_exists == 'replace':
#             sql = f"DELETE FROM {schema}.{table_name}"
#             engine.execute(sql)
#             print('SQL table has been cleaned up successfully.')
#         else:
#             pass
#     else:
#         df.head(1).to_sql(table_name, schema=schema, index=False, con=engine, dtype=dtype)

#     if not keep_csv:
#         os.remove(filepath)

In [ ]:
#dateTime (optional)string
#Format - date-time (as date-time in RFC3339). Departure date / time for the search. defaults to server time (Europe/Amsterdam)

# Appendix

## Other APIs

In [ ]:
# #disruptions
# headers = {
#     # Request headers
#     'Ocp-Apim-Subscription-Key': '%s' % key ,
# }

# params = urllib.parse.urlencode({
#     # Request parameters
#     'type': '{string}',
#     'actual': '{boolean}',
#     'lang': 'en',
# })

# try:
#     conn = http.client.HTTPSConnection('gateway.apiportal.ns.nl')
#     conn.request("GET", "/public-reisinformatie/api/v2/disruptions?%s" % params, "{body}", headers)
#     response = conn.getresponse()
#     disruption = response.read()
#     #print(data)
#     conn.close()
# except Exception as e:
#     print("[Errno {0}] {1}".format(e.errno, e.strerror))


In [ ]:
# station = "amsterdamzuid"

In [ ]:
# #Disruption station
# headers = {
#     # Request headers
#     'Ocp-Apim-Subscription-Key': '%s' % key ,
# }

# params = urllib.parse.urlencode({
# })

# try:
#     conn = http.client.HTTPSConnection('gateway.apiportal.ns.nl')
#     conn.request("GET", "/public-reisinformatie/api/v2/disruptions/station/%s" % station, "{body}", headers)
#     response = conn.getresponse()
#     disrupt_station = response.read()
#     #print(data)
#     conn.close()
# except Exception as e:
#     print("[Errno {0}] {1}".format(e.errno, e.strerror))


In [ ]:
# # This doesn't work yet
# uid_add = "http://webservices.ns.nl/ns-api-stations-v2?_ga=2.141585782.518451146.1577077875-2005063189.1572857950"
# key = os.getenv("NS_KEY")

# headers = {
#     # Request headers
#     'Ocp-Apim-Subscription-Key': '%s' % key,
# } 
# try:
#     conn = http.client.HTTPSConnection('gateway.apiportal.ns.nl')
#     conn.request("GET", uid_add, "{body}", headers)
#     response = conn.getresponse()
#     uid = response.read()
#     #print(data)
#     conn.close()
# except Exception as e:
#     print("[Errno {0}] {1}".format(e.errno, e.strerror))

## Code used to explore json file

In [ ]:
# NeatNamespace(d)

In [ ]:
# trips = d['trips'][0]

# NeatNamespace(trips)

legs = d['trips'][0]['legs'][0]

len(legs)

NeatNamespace(legs)

len(legs)

#d

## Explore dictionary in old way

df = df.reset_index()

df_lcopy = df['legs'].copy()

df_lcopy

frames = []
for i in range(len(df_lcopy)):    
    frames.append(json_normalize(df_lcopy[i]))

df_leg = pd.concat(frames, sort='True')

df_leg

#df_unpack[df_unpack['name']==train_id]

#df_unpack.info()

#df_trips.query('transfers == 0')

#df_leg['destination.notes'].values
#df_leg['notes'].values

columns_select = ['destination.prognosisType', 'origin.name','destination.name', 'cancelled','origin.plannedDateTime', 'origin.actualDateTime', 'destination.plannedDateTime','punctuality']
all_columns = df_leg.columns

intersection = set(columns_select) & set(all_columns)

df_an = df_leg[intersection]

date_time_col = intersection.islike('%Date%')

df_an

df_an = pd.to_datetime(df_an['origin.plannedDateTime'])


# import smtplib

# def send_email(status, delayed_min, url_to_html ):
#     gmail_user = os.getenv('GMAIL_USER')
#     gmail_password = os.getenv('GMAIL_PWD')
#     send_to = [i.strip() for i in list(os.getenv("SEND_TO").split(";"))]

#     sent_from = gmail_user
#     to = send_to
#     subject = 'Train '+status+'!'
#     body = 'test'

#     email_text = """\
#     From: %s
#     To: %s
#     Subject: %s
#     Body:
#     %s
#     """ % (sent_from,'b.luo.pmp@gmail.com' , subject, body) # "; ".join(to)
#     server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
#     server.ehlo()
#     server.login(gmail_user, gmail_password)
#     server.sendmail(sent_from, to, email_text)
#     server.close()
    
# #     try:
# #         server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
# #         server.ehlo()
# #         server.login(gmail_user, gmail_password)
# #         server.sendmail(sent_from, to, email_text)
# #         server.close()

# #         print('Email sent!')
# #     except:
# #         print('Something went wrong...')